In [1]:
import os

os.environ['ML_FLOW_TRACKING_URI'] = 'https://dagshub.com/satwiksahoojob/data_science_project.mlflow'
os.environ['ML_FLOW_TRACKING_USERNAME'] = 'satwiksahoojob'

os.environ['ML_FLOW_TRACKING_PASSWORD'] =  '46df081cd1c771ae5c58838602b17d7d7c7de746'

In [4]:
import os 

%pwd

'/Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project'

In [18]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class modelevaluationconfig:
    root_dir : Path
    
    test_data_path : Path
    model_path: Path
    all_params : dict
    metric_file_name : Path
    target_column : str
    mlflow_uri : str

In [19]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml , create_directories

class configmanager:
    def __init__(self, config_filepath = config_file_path , params_filepath = param_file_path , schema_filepath = schema_file_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> modelevaluationconfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        create_directories([config.root_dir])    
        schema  = self.schema.TARGET_COLUMN


        modelevaluation_config = modelevaluationconfig(

            root_dir =  config.root_dir,
            
            test_data_path = config.test_data_path , 
            model_path =  config.model_path ,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = 'https://dagshub.com/satwiksahoojob/data_science_project.mlflow' 

        )

        return modelevaluation_config           

In [20]:
import os
import pandas as pd
from src.datascience import logger
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score
from urllib.parse import urlparse
import joblib
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.linear_model import ElasticNet
import json
from src.datascience.utils.common import save_json


class model_evaluation:

    def __init__(self, config: modelevaluationconfig):
        self.config = config

    def evaluation_metric(self, actual, pred):

        rmse = np.sqrt(mean_squared_error(actual , pred))
        mae = mean_absolute_error(actual , pred)
        r2  = r2_score(actual , pred)

        return rmse , mae , r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x  =test_data.drop([self.config.target_column] ,axis=1)
        test_y  =test_data[self.config.target_column]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme 


        with mlflow.start_run():  # 🟢 Start MLflow run

    # Initialize and train model
        #  model = ElasticNet(alpha=self.params.alpha, l1_ratio=self.params.l1_ratio, random_state=42)
        #  model.fit(X_train, y_train)

    # Predict and evaluate
         preds = model.predict(test_x)
        #  mse = mean_squared_error(test_y, preds)
        #  r2 = r2_score(test_y, preds)
         (rmse , mae , r2) = self.evaluation_metric(test_y , preds )

         scores = { 'rmse':rmse ,'mae': mae ,'r2' : r2}
         save_json(path = Path(self.config.metric_file_name), data = scores)
    # Log parameters, metrics, and model
         mlflow.log_param('all params',self.config.all_params)
         mlflow.log_metric("mae", mae)
         mlflow.log_metric("r2", r2)
         mlflow.log_metric("rmse", rmse)

         if tracking_uri_type_store != 'file':
            mlflow.sklearn.log_model(model, "model" , registered_model_name='ElasticNet model')
         else:    
           print("Run logged in MLflow.")




In [21]:
try:
    config = configmanager() 
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = model_evaluation(config=model_evaluation_config) 
    model_evaluation.log_into_mlflow()



   
  

except Exception as e:
   raise e    

[DEBUG] Trying to load YAML from: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project/config/config.yaml | Exists: True
[2025-07-02 12:54:15,247 : INFO : common : YAML file: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project/config/config.yaml loaded successfully.]
[DEBUG] Trying to load YAML from: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project/params.yaml | Exists: True
[2025-07-02 12:54:15,249 : INFO : common : YAML file: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project/params.yaml loaded successfully.]
[DEBUG] Trying to load YAML from: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/data_science_project/schema.yaml | Exists: True
[2025-07-02 12:54:15,251 : INFO : common : YAML file: /Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/1st_project/